# Install geotable, GDAL

```
$ brew install gdal libkml
$ pip install GDAL geotable
```

# Data download

[Map](https://www.google.com/maps/d/u/0/viewer?mid=1hQhJDhsE87Iu9BJOln-EnveGbow&ll=35.07345181763887%2C138.8560247350858&z=12)

[KML Download](https://www.google.com/maps/d/u/0/kml?forcekml=1&mid=1hQhJDhsE87Iu9BJOln-EnveGbow&lid=Cs8qfjzbvoQ)

In [1]:
!wget -cO - 'https://www.google.com/maps/d/u/0/kml?forcekml=1&mid=1hQhJDhsE87Iu9BJOln-EnveGbow&lid=Cs8qfjzbvoQ' > ../data/machiaruki.kml

--2023-04-12 12:04:06--  https://www.google.com/maps/d/u/0/kml?forcekml=1&mid=1hQhJDhsE87Iu9BJOln-EnveGbow&lid=Cs8qfjzbvoQ
Resolving www.google.com (www.google.com)... 142.251.42.132
Connecting to www.google.com (www.google.com)|142.251.42.132|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.google.com/maps/d/kml?forcekml=1&mid=1hQhJDhsE87Iu9BJOln-EnveGbow&lid=Cs8qfjzbvoQ [following]
--2023-04-12 12:04:06--  https://www.google.com/maps/d/kml?forcekml=1&mid=1hQhJDhsE87Iu9BJOln-EnveGbow&lid=Cs8qfjzbvoQ
Reusing existing connection to www.google.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/xml]
Saving to: ‘STDOUT’

-                       [ <=>                ] 253.98K  --.-KB/s    in 0.06s   

2023-04-12 12:04:06 (3.86 MB/s) - written to stdout [260077]



# Geotable

In [2]:
import os
import geotable

In [63]:
data_kml = os.path.join(os.path.abspath('..'), 'data/machiaruki.kml')
t = geotable.load(data_kml)
t

,Name,Description,geometry_object,geometry_layer,geometry_proj4
0,ゲーマーズ沼津店,"<img src=""https://doc-00-9o-mymaps.googleuserc...",POINT Z (138.856807 35.10157 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
1,安田屋旅館,"<img src=""https://doc-0c-9o-mymaps.googleuserc...",POINT Z (138.897573 35.020424 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
2,和洋菓子松月,"<img src=""https://doc-14-9o-mymaps.googleuserc...",POINT Z (138.897925 35.022259 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
3,SUN!SUN!サンシャインCAFE,"<img src=""https://doc-0g-9o-mymaps.googleuserc...",POINT Z (138.8597404 35.1019932 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
4,オーシャン ビュー フジミ,"<img src=""https://doc-10-9o-mymaps.googleuserc...",POINT Z (138.7858531 35.0249211 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
...,...,...,...,...,...
112,グランマ シーサイド店,"<img src=""https://doc-0c-9o-mymaps.googleuserc...",POINT Z (138.8860146 35.0481496 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
113,テレビのイマイ,"<img src=""https://doc-14-9o-mymaps.googleuserc...",POINT Z (138.8578183 35.0986286 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
114,戸田観光協会,"<img src=""https://doc-0s-9o-mymaps.googleuserc...",POINT Z (138.7772204 34.9729703 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
115,モスバーガーららぽーと沼津店,"<img src=""https://doc-14-9o-mymaps.googleuserc...",POINT Z (138.8419064 35.1204196 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs


# Data Cleaning

In [4]:
import re

In [64]:
def df_clean(df):
    # drop unneeded columns
    df = df.drop(columns=['geometry_layer','geometry_proj4'])
    
    # regex 'Description' and split into new columns
    df['img'] = df['Description'].apply(lambda x: re.findall('<img src=.*?/>', x)[0])
    df['member'] = df['Description'].apply(lambda x: re.findall('メンバー／(.*)<br>住', x)[0])
    df['address'] = df['Description'].apply(lambda x: re.findall('住所／(.*)<br>営', x)[0])
    df['hours'] = df['Description'].apply(lambda x: re.findall('営業時間／(.*)<br>定', x))
    df['holidays'] = df['Description'].apply(lambda x: re.findall('定休日／(.*)', x))
    
    return df

In [65]:
t_clean = df_clean(t)
t_clean

,Name,Description,geometry_object,img,member,address,hours,holidays
0,ゲーマーズ沼津店,"<img src=""https://doc-00-9o-mymaps.googleuserc...",POINT Z (138.856807 35.10157 0),"<img src=""https://doc-00-9o-mymaps.googleuserc...",津島善子,沼津市添地町72青秀ビル1階,[平日 11:00～20:00<br>土日祝 10:0～20:00],[なし]
1,安田屋旅館,"<img src=""https://doc-0c-9o-mymaps.googleuserc...",POINT Z (138.897573 35.020424 0),"<img src=""https://doc-0c-9o-mymaps.googleuserc...",高海千歌,沼津市内浦三津19,[12:00～20:00],[不定休]
2,和洋菓子松月,"<img src=""https://doc-14-9o-mymaps.googleuserc...",POINT Z (138.897925 35.022259 0),"<img src=""https://doc-14-9o-mymaps.googleuserc...",国木田花丸,沼津市内浦88-36,[10:00～17:00],[火・水曜日]
3,SUN!SUN!サンシャインCAFE,"<img src=""https://doc-0g-9o-mymaps.googleuserc...",POINT Z (138.8597404 35.1019932 0),"<img src=""https://doc-0g-9o-mymaps.googleuserc...",黒澤ダイヤ,沼津市大手町3-1-1,[平日 11:00～20:30<br>土日祝 10:00～20:30],[なし]
4,オーシャン ビュー フジミ,"<img src=""https://doc-10-9o-mymaps.googleuserc...",POINT Z (138.7858531 35.0249211 0),"<img src=""https://doc-10-9o-mymaps.googleuserc...",松浦果南,沼津市西浦江梨993,[平日 10:30～14:30<br>土日祝 10:00～14:30],[なし]
...,...,...,...,...,...,...,...,...
112,グランマ シーサイド店,"<img src=""https://doc-0c-9o-mymaps.googleuserc...",POINT Z (138.8860146 35.0481496 0),"<img src=""https://doc-0c-9o-mymaps.googleuserc...",松浦果南,沼津市獅子浜22-27,[ 10:00 ～ 18:30（L.O.17:00）],[火曜日・第三水曜日<br>https://sweets-grandma.com/seasi...
113,テレビのイマイ,"<img src=""https://doc-14-9o-mymaps.googleuserc...",POINT Z (138.8578183 35.0986286 0),"<img src=""https://doc-14-9o-mymaps.googleuserc...",高海千歌,沼津市大手町4-6-2,[ 10:00 ～ 18:00],[水曜日・日曜日]
114,戸田観光協会,"<img src=""https://doc-0s-9o-mymaps.googleuserc...",POINT Z (138.7772204 34.9729703 0),"<img src=""https://doc-0s-9o-mymaps.googleuserc...",黒澤ルビィ,沼津市戸田289-12,[ 9:00～17:00 ],[年中無休（年末年始を除く）<br>https://www.deep-heda.com]
115,モスバーガーららぽーと沼津店,"<img src=""https://doc-14-9o-mymaps.googleuserc...",POINT Z (138.8419064 35.1204196 0),"<img src=""https://doc-14-9o-mymaps.googleuserc...",桜内梨子,沼津市東椎路字東荒３０１-３ 3Fフードコート,[],[]


In [66]:
x = t.Description[115]
y = t.Description[0]

In [81]:
x

'<img src="https://doc-14-9o-mymaps.googleusercontent.com/untrusted/hostedimage/o2fbn585vcrt3ao71o6a0j9c34/v076d43m9nmcgv1me99dfei8nk/1681268400000/hzzDqczHXdAc-TMWLZLAFJCdFFMq9JPf/*/6ACtvi-GKz0Tlax6gYar5HCN7GAtcNjd-ITc9P9RdvQM4kZRSg9cnX79N5nIgsIo56NyYYT3Ah4sT_t_Nht12tV8V6H0qC6MI7EwSu__Kfmm5uOvO97QA0iIV6DtTxufhI-78bW31f1MUIHgOzXn8gDQhusal3tc-P2ftJBlf4slJwnBxqFhYtQ68glMMbeX-?fife" height="200" width="auto" /><br><br>メンバー／桜内梨子<br>住所／沼津市東椎路字東荒３０１-３ 3Fフードコート<br>営業時間／平日（※祝日を除く）10:00 ~ 20:00<br> 土日祝日10:00 ~ 21:00<br>※ご来館の際には最新の営業時間をご確認のうえお越しください。<br>https://www.mos.jp/shop/detail/?shop_cd=05290'

In [96]:
re.findall('営業時間／(.*)', x)

['平日（※祝日を除く）10:00 ~ 20:00<br> 土日祝日10:00 ~ 21:00<br>※ご来館の際には最新の営業時間をご確認のうえお越しください。<br>https://www.mos.jp/shop/detail/?shop_cd=05290']